In [1]:
import psycopg2, pandas as pd, numpy as np

In [2]:
def create_database():
    conn = psycopg2.connect("host = 127.0.0.1 dbname= postgres user= postgres password= ps")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    cur.execute("DROP DATABASE IF EXISTS accounts")
    cur.execute("CREATE DATABASE accounts")

    conn.close()

    conn = psycopg2.connect("host = 127.0.0.1 dbname= accounts user= postgres password= ps")
    cur = conn.cursor()

    return cur, conn

In [3]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)

In [4]:
AccountsCountry = pd.read_csv("C:/Users/hp/Wealth dataset/Wealth-AccountsCountry.csv")

In [5]:
AccountsCountry.head()

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,IBRD,NaN,Armenian dram,2020 (expected),"Demographic and Health Survey, 2015/16",NaN,...,"Integrated household survey (IHS), 2016",Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia,Armenia
3,AUS,Commonwealth of Australia,High income,East Asia & Pacific,NaN,NaN,Australian dollar,2016,NaN,Fiscal year end: June 30; reporting period for...,...,"Expenditure survey/budget survey (ES/BS), 2010",Yes,2015-2016,2013.0,2018.0,2013.0,AU,AU,Australia,Australia
4,AUT,Republic of Austria,High income,Europe & Central Asia,NaN,Euro area,Euro,2011. Population figures compiled from adminis...,NaN,A simple multiplier is used to convert the nat...,...,"Income survey (IS), 2015",Yes,2010,2014.0,2018.0,2010.0,AT,AT,Austria,Austria


In [6]:
AccountsCountry_clean = AccountsCountry[['Code', 'Short Name', 'Table Name',
                                         'Long Name', 'Currency Unit']].copy()
AccountsCountry_clean.rename(columns={'Code': 'Country Code'}, inplace=True)

In [7]:
AccountsCountry_clean.head()

,Country Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


In [8]:
AccountsData = pd.read_csv("C:/Users/hp/Wealth dataset/Wealth-AccountData.csv")
AccountsData.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3355.549,2868.997,3013.391,3259.893,3424.007,...,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3072.504,3178.417,3312.927
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6353.737,5466.692,5901.072,6492.73,6872.019,...,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,17817.1,18445.94,19245.19
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4737.857,4059.321,4191.445,4536.972,4695.071,...,3.529144e+03,3.613606e+03,3.603694e+03,3.683810e+03,3.696744e+03,3.719608e+03,3.767628e+03,3872.142,3992.188,4161.137


In [9]:
AccountsData.columns


Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

In [10]:
AccountsData = AccountsData[['Country Name', 'Country Code', 'Series Name',
                       'Series Code', '1995 [YR1995]', '2000 [YR2000]',
                       '2005 [YR2005]', '2010 [YR2010]', '2015 [YR2015]']]
AccountsData.rename(columns={'Series Name': 'Indicator Name',
                   'Series Code': 'Indicator Code'}, inplace=True)
AccountsData.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1995 [YR1995],2000 [YR2000],2005 [YR2005],2010 [YR2010],2015 [YR2015]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.370000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.559371e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3424.007,2.222115e+03,2.889150e+03,2.987545e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6872.019,1.168935e+04,1.578992e+04,1.730701e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4695.071,2.886736e+03,3.613606e+03,3.767628e+03


In [11]:
numeric_cols = ['1995 [YR1995]', '2000 [YR2000]', '2005 [YR2005]', '2010 [YR2010]', '2015 [YR2015]']
AccountsData[numeric_cols] = AccountsData[numeric_cols].replace('..', np.nan).astype(float)

In [12]:
AccountsSeries = pd.read_csv("C:/Users/hp/Wealth dataset/Wealth-AccountSeries.csv")
AccountsSeries.head()

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN


In [13]:
AccountsSeries.columns

Index(['Code', 'Indicator Name', 'Long definition', 'Source', 'Topic',
       'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name'],
      dtype='object')

In [14]:
AccountsSeries = AccountsSeries[['Code', 'Topic', 'Indicator Name', 'Long definition']]

In [15]:
AccountsSeries.head()

,Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",Human capital is computed as the present value...
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",Human capital is computed as the present value...
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...


In [16]:
cur, conn = create_database()

In [17]:
songplay_table_create = ("""CREATE TABLE IF NOT EXISTS accountscountry(
                                Country_Code VARCHAR PRIMARY KEY,
                                Short_Name VARCHAR,
                                Table_Name VARCHAR,
                                Long_Name VARCHAR,
                                Currency_Unit VARCHAR
                                )""")

In [18]:
cur.execute(songplay_table_create)
conn.commit()

In [19]:
accounts_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(
                                Country_Name VARCHAR,
                                Country_Code VARCHAR,
                                Indicator_Name VARCHAR,
                                Indicator_Code VARCHAR,
                                Year_1995 numeric,
                                Year_2000 numeric,
                                Year_2005 numeric, 
                                Year_2010 numeric,
                                Year_2015 numeric
                                )""")
cur.execute(accounts_data_table_create)
conn.commit()

In [20]:
accountseries_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountseries(
                                   Series_Code VARCHAR,
                                   Topic VARCHAR,
                                   Indicator_Name VARCHAR,
                                   Long_definition VARCHAR
                                   )""")
cur.execute(accountseries_data_table_create)
conn.commit()

In [21]:
accounts_country_table_insert = """
INSERT INTO accountscountry (
    Country_Code,
    Short_Name,
    Table_Name,
    Long_Name,
    Currency_Unit
)
VALUES (%s, %s, %s, %s, %s)
ON CONFLICT (Country_Code) DO NOTHING;
"""

In [22]:
for i, row in AccountsCountry_clean.iterrows():
    cur.execute(accounts_country_table_insert, list(row))


In [23]:
conn.commit()

In [24]:
accounts_data_table_insert = """
INSERT INTO accountsdata (
    Country_Name,
    Country_Code,
    Indicator_Name,
    Indicator_Code,
    Year_1995,
    Year_2000,
    Year_2005, 
    Year_2010,
    Year_2015
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

for i, row in AccountsData.iterrows():
    cur.execute(accounts_data_table_insert, list(row))

In [25]:
conn.commit()

In [26]:
accountseries_table_insert = """
INSERT INTO accountseries (
    Series_Code,
    Topic,
    Indicator_Name,
    Long_definition
)
VALUES (%s, %s, %s, %s);
"""

In [27]:
for i, row in AccountsSeries.iterrows():
    cur.execute(accountseries_table_insert, list(row))

In [28]:
conn.commit()